<a href="https://colab.research.google.com/github/kumar987789/daily-updates/blob/main/pyspark_multidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PySpark Practice Notebook
Experimenting further with Spark Dataframes

In [ ]:
#Create CSV files for the following Datasets

patients
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78

visits
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15

In [1]:
%%writefile patients.csv
patientId,firstName,lastName,age
101,Alice,Smith,30
102,Bob,Johnson,45
103,Charlie,Williams,50
104,John,Smith,78


Writing patients.csv


In [2]:
%%writefile visits.csv
visitId,patientId,visitDuration
1,101,15
2,101,30
3,102,45
4,102,30
5,104,20
6,103,60
7,103,50
8,104,45
9,,45
10,101,30
11,103,40
12,,30
13,104,25
15,102,15


Writing visits.csv


In [5]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=c26a04e8571987e89778dcd7324295e2071843fe600672bace0ff5d9dc2cda6c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [8]:
#Import necessary spark libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *




In [9]:
#Create a SparkSession
spark=SparkSession.builder.appName('nani').getOrCreate()


In [17]:
#Create dataframes from CSV files using enforced Schema
#Schems Directive [Names as string, everything else as integers]

df=spark.read.option('header','True').csv('patients.csv')
df1=spark.read.option('header','True').csv('visits.csv')


In [53]:
#Display all the valid visits
df1.show()


#Fetch the total number of valid visits
df1.count()



+-------+---------+-------------+
|visitId|patientId|visitDuration|
+-------+---------+-------------+
|      1|      101|           15|
|      2|      101|           30|
|      3|      102|           45|
|      4|      102|           30|
|      5|      104|           20|
|      6|      103|           60|
|      7|      103|           50|
|      8|      104|           45|
|      9|     NULL|           45|
|     10|      101|           30|
|     11|      103|           40|
|     12|     NULL|           30|
|     13|      104|           25|
|     15|      102|           15|
+-------+---------+-------------+



14

In [16]:
#Get the total hours of patient visit

df.agg({'visitDuration':'sum'}).withColumnRenamed('sum(visitDuration)','total_hours').show()





+-----------+
|total_hours|
+-----------+
|      480.0|
+-----------+



In [25]:
#Fetch the top two most visited patients on the basis of occurances
x=df.join(df1,'patientId')
x.groupBy('patientId').count().orderBy('count',ascending=False).show(2)



+---------+-----+
|patientId|count|
+---------+-----+
|      101|    3|
|      104|    3|
+---------+-----+
only showing top 2 rows



In [28]:
#Add a new column patientFullName & display complete dataframe
df.withColumn('patientFullName',concat(col('firstName'),lit(' '),col('lastName'))).show()
df.show()

+---------+---------+--------+---+----------------+
|patientId|firstName|lastName|age| patientFullName|
+---------+---------+--------+---+----------------+
|      101|    Alice|   Smith| 30|     Alice Smith|
|      102|      Bob| Johnson| 45|     Bob Johnson|
|      103|  Charlie|Williams| 50|Charlie Williams|
|      104|     John|   Smith| 78|      John Smith|
+---------+---------+--------+---+----------------+

+---------+---------+--------+---+
|patientId|firstName|lastName|age|
+---------+---------+--------+---+
|      101|    Alice|   Smith| 30|
|      102|      Bob| Johnson| 45|
|      103|  Charlie|Williams| 50|
|      104|     John|   Smith| 78|
+---------+---------+--------+---+



In [41]:
#Display the name of patients with maximum visiting hours
x=df.join(df1,'patientId')
x.groupBy('patientId').agg({'visitDuration':'sum'}).orderBy('sum(visitDuration)',ascending=False).show(1)


+---------+------------------+
|patientId|sum(visitDuration)|
+---------+------------------+
|      103|             150.0|
+---------+------------------+
only showing top 1 row



In [49]:
#Find the patient with maximum visiting hours in a single visit

x=df.join(df1,'patientId')
x.orderBy('visitDuration',ascending=False).show(1)




+---------+---------+--------+---+-------+-------------+
|patientId|firstName|lastName|age|visitId|visitDuration|
+---------+---------+--------+---+-------+-------------+
|      103|  Charlie|Williams| 50|      6|           60|
+---------+---------+--------+---+-------+-------------+
only showing top 1 row



In [51]:
#Show the patient info according to age-seniority
#df.orderBy('age',ascending=False).show()
df.sort('age',ascending=False).show()

+---------+---------+--------+---+
|patientId|firstName|lastName|age|
+---------+---------+--------+---+
|      104|     John|   Smith| 78|
|      103|  Charlie|Williams| 50|
|      102|      Bob| Johnson| 45|
|      101|    Alice|   Smith| 30|
+---------+---------+--------+---+

